In [1]:
import logging
from pathlib import Path
import os
import sys
import mne
import torch
import numpy as np
import bm
from bm import play
from bm.train import main
from bm.events import Word
from matplotlib import pyplot as plt
from IPython import display as disp

mne.set_log_level(False)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
os.chdir(main.dora.dir.parent)
os.environ['NO_DOWNLOAD'] = '1'
from tqdm import tqdm
import IPython.display as ipd

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'play' from 'bm' (unknown location)

In [2]:
sig='342eaad6'

In [3]:
solver = play.get_solver_from_sig(sig) 

INFO:bm.play:Loading solver from XP 342eaad6. Overrides used: ['model=clip_conv', 'dset.selections=["gwilliams2022"]', 'dset.bandpass=true', 'dset.bandpass_high=0.1', 'dset.bandpass_lower=40.0', 'seed=2038', 'dset.tmin=-0.75', 'dset.tmax=3.25']
{'wandb': {'use_wandb': True, 'project': 'brainmagick', 'group': 'brainmagick-group'}, 'num_prints': 5, 'device': 'cuda:1', 'num_workers': 5, 'verbose': 0, 'show': 0, 'download_only': False, 'slurm': {'mem_per_gpu': 100, 'time': 4320}, 'continue_sig': None, 'continue_best': True, 'seed': 2038, 'dummy': None, 'cache': '/home/zubrikhina/brainmagick_MICCAI/cache', 'features_models': './features_models', 'early_stop_patience': 10, 'eval_every': 1, 'eval_train_set': False, 'optim': {'name': 'adam', 'shuffle': False, 'lr': 0.0003, 'beta2': 0.999, 'epochs': 200, 'batch_size': 256, 'loss': 'clip', 'weight_decay': 0.0, 'use_weighting': False, 'max_batches': 1200, 'svd': 0.0, 'negatives': None, 'negative_pool_size': None}, 'clip': {'linear': None, 'twin':

/home/zubrikhina/anaconda3/envs/bm/lib/python3.8/site-packages/flashy/loggers/tensorboard.py:47: UserWarning: tensorboard package was not found: use pip install tensorboard
  warnings.warn("tensorboard package was not found: use pip install tensorboard")
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [4]:
## SegmentDataset

In [5]:
segment =solver.datasets.train.datasets[0]

In [25]:
segment.events.columns

Index(['story', 'story_uid', 'sound_id', 'kind', 'start', 'sound', 'duration',
       'filepath', 'phoneme', 'sequence_id', 'condition', 'word_index',
       'speech_rate', 'voice', 'pronounced', 'word', 'language', 'modality',
       'word_sequence', 'phoneme_id', 'offset', 'uid'],
      dtype='object')

In [27]:
segment.events.loc[segment.events.kind =='sound'][["start", "offset"]]

,start,offset
0,23.506,0.00
97,29.756,6.25
259,40.096,16.59
497,68.416,44.91
584,79.166,55.66
661,91.926,68.42
723,98.206,74.70
817,109.876,86.37
889,127.185,0.00
925,131.415,4.23


In [14]:
segment.events.columns

Index(['story', 'story_uid', 'sound_id', 'kind', 'start', 'sound', 'duration',
       'filepath', 'phoneme', 'sequence_id', 'condition', 'word_index',
       'speech_rate', 'voice', 'pronounced', 'word', 'language', 'modality',
       'word_sequence', 'phoneme_id', 'offset', 'uid'],
      dtype='object')

In [15]:
segment._get_bounds_times(0)

(30.25, 34.25833333333333)

In [16]:
feature_data, feature_mask, events = segment._get_feature(0)

In [17]:
events

[DataSlice(start=30.25, duration=4.008333333333333, modality=None, language=None, sample_rate=120.0),
 Sound(start=29.756, duration=10.339999999999996, modality='audio', language='english', filepath='/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/lw1_0.wav', offset=6.25),
 Word(start=29.896, duration=0.3900000000000005, modality='audio', language='english', word='gentle', word_index=1, word_sequence='The gentle constant breeze of recycled air from the vent above blew an annoying hair against her nose but she ignored it'),
 Word(start=30.286, duration=0.4199999999999999, modality='audio', language='english', word='constant', word_index=2, word_sequence='The gentle constant breeze of recycled air from the vent above blew an annoying hair against her nose but she ignored it'),
 Word(start=30.706, duration=0.2099999999999999, modality='audio', language='english', word='breeze', word_index=3, word_sequence='The gentle constant breeze of recycled air from the v

In [18]:
from bm.features.audio import _extract_wav_part

In [19]:
wav, sr =_extract_wav_part('/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/lw1_0.wav', 30.25, 34.25833333333333)

In [20]:
sr

22050.0

In [21]:
wav.numpy()

array([[-0.335083  , -0.32199097, -0.29586792, ...,  0.10470581,
         0.16851807,  0.19934082]], dtype=float32)

In [22]:
ipd.Audio(data=wav.numpy(), autoplay=True, rate=sr)

In [23]:
wav2, sr =_extract_wav_part('/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/lw1_0.wav', 6.25, 10.339999999999996+6.25)

In [24]:
ipd.Audio(data=wav2.numpy(), autoplay=True, rate=sr)

In [28]:
def _get_one_segments_and_vocabs(solver):
    from scripts.run_eval_probs import _get_extra_info
    split ='train'
    segments,vocab, estimates,outputs,features_masks, reject_masks = [], [],[], [],[], []
    dset = getattr(solver.datasets, split)
    loader = solver.make_loader(dset, shuffle=False)
    test_features = solver.datasets.test.datasets[0].features
    for idx, batch in tqdm(enumerate(loader)):
        #features = test_features.extract_features(batch.features, solver.used_features.keys())
        with torch.no_grad():
            estimate, output, features_mask, reject_mask = solver._process_batch(batch)
            data,  words, word_segs= _get_extra_info(batch, solver.args.dset.sample_rate)
        if idx>1:
            break
    return batch, estimate, output, word_segs,vocab

In [29]:
batch, estimate, output, word_segs,vocab = _get_one_segments_and_vocabs(solver)

2it [00:10,  5.50s/it]


In [30]:
word_segs[0]

'of his eye Two forts stood on the playground and a hot'

In [34]:
batch._event_lists[0]

[DataSlice(start=96.25, duration=4.00833333333334, modality=None, language=None, sample_rate=120.0),
 Sound(start=94.165, duration=8.849999999999994, modality='audio', language='english', filepath='/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/cable_spool_fort_0.wav', offset=85.92),
 Word(start=96.185, duration=0.1299999999999954, modality='audio', language='english', word='of', word_index=9, word_sequence='Chad checked the other fort out of the corner of his eye'),
 Word(start=96.315, duration=0.1400000000000005, modality='audio', language='english', word='his', word_index=10, word_sequence='Chad checked the other fort out of the corner of his eye'),
 Word(start=96.455, duration=0.0900000000000034, modality='audio', language='english', word='eye', word_index=11, word_sequence='Chad checked the other fort out of the corner of his eye'),
 Word(start=97.735, duration=0.2100000000000079, modality='audio', language='english', word='Two', word_index=0, word_s

In [37]:
wav3, sr =_extract_wav_part('/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/cable_spool_fort_0.wav', 85.92, 8.849999999999994+85.92)

In [38]:
ipd.Audio(data=wav3.numpy(), autoplay=True, rate=sr)

In [39]:
word_segs

array(['of his eye Two forts stood on the playground and a hot',
       'Two forts stood on the playground and a hot bare battlefield',
       'Two forts stood on the playground and a hot bare battlefield',
       'Two forts stood on the playground and a hot bare battlefield separated',
       'stood on the playground and a hot bare battlefield separated them',
       'playground and a hot bare battlefield separated them',
       'massive rusted A frame swing marked the border though nobody was swinging',
       'rusted A frame swing marked the border though nobody was swinging in the hot',
       'rusted A frame swing marked the border though nobody was swinging in the hot sun',
       'frame swing marked the border though nobody was swinging in the hot sun',
       'swing marked the border though nobody was swinging in the hot sun',
       'the border though nobody was swinging in the hot sun',
       'though nobody was swinging in the hot sun',
       'was swinging in the hot sun', 

In [40]:
np.where(word_segs == "Roy")

(array([193, 194, 195, 196, 197, 198, 199, 200, 208, 209, 210, 211, 212,
        213, 214, 215, 216, 221, 222, 223, 224, 225, 226, 227, 228, 229,
        231]),)

In [41]:
batch._event_lists[193]

[DataSlice(start=309.25, duration=4.008333333333326, modality=None, language=None, sample_rate=120.0),
 Sound(start=302.347, duration=33.97000000000003, modality='audio', language='english', filepath='/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/cable_spool_fort_2.wav', offset=44.629999999999995),
 Word(start=312.777, duration=0.3999999999999986, modality='audio', language='english', word='Roy', word_index=10, word_sequence='Pop said The doctors have put something very special in Roy')]

In [42]:
batch._event_lists[194]

[DataSlice(start=309.75, duration=4.008333333333326, modality=None, language=None, sample_rate=120.0),
 Sound(start=302.347, duration=33.97000000000003, modality='audio', language='english', filepath='/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/cable_spool_fort_2.wav', offset=44.629999999999995),
 Word(start=312.777, duration=0.3999999999999986, modality='audio', language='english', word='Roy', word_index=10, word_sequence='Pop said The doctors have put something very special in Roy')]

In [45]:
batch.subject_index[194]

tensor(0)

In [46]:
batch.subject_index[193]

tensor(0)

In [49]:
scores = torch.einsum("bct,oct->bo", output, output)

In [53]:
scores[193,:].sort()

torch.return_types.sort(
values=tensor([ -1040.4744,  -1040.4744,  -1040.4744,  -1040.4744,  -1040.4744,
         -1040.4744,  -1040.4744,  -1040.4744,  -1040.4744,  -1040.4744,
         -1040.4744,   1350.7147,   1561.6619,  12609.6328,  16526.9629,
         28826.8555,  30469.7617,  40457.1289,  45959.9258,  59124.7500,
         60368.9180,  65118.1836,  65172.9570,  69158.4375,  71125.6016,
         76552.3594,  80116.0234,  83802.4766,  83947.8984,  84267.6484,
         84426.8047,  86211.3438,  86597.4219,  87915.7500,  87964.5000,
         92221.4766,  92631.1953,  95835.1016,  95939.9219,  95954.3984,
         98119.2500,  98282.3750,  99386.4062,  99896.7734,  99965.7812,
        100004.1328, 100412.0703, 101469.5312, 101528.2891, 101708.5859,
        101717.2578, 101835.3047, 102027.8438, 102572.5938, 102902.0781,
        103422.2266, 103966.6406, 104395.5391, 104421.0312, 105096.6484,
        105100.4219, 105231.1953, 105530.3359, 105791.7266, 105837.0703,
        106018.4219

In [56]:
scores_f = torch.einsum("bct,oct->bo", batch.features, batch.features)

In [57]:
scores_f[193,:].sort()

torch.return_types.sort(
values=tensor([       0.0000,        0.0000,        0.0000,        0.0000,
               0.0000,        0.0000,        0.0000,        0.0000,
               0.0000,        0.0000,        0.0000,    80851.1016,
           88165.3594,   476109.1875,   608505.1875,  1045624.2500,
         1112549.2500,  1472742.8750,  1667045.0000,  2186037.5000,
         2320733.5000,  2489101.5000,  2692286.2500,  2787718.2500,
         2972300.7500,  3008604.2500,  3025721.7500,  3185151.5000,
         3281116.5000,  3300858.2500,  3322161.0000,  3362557.7500,
         3393898.2500,  3434612.2500,  3444947.2500,  3474796.7500,
         3655056.2500,  3692100.2500,  3740408.7500,  3794827.2500,
         3811069.7500,  3828682.2500,  3835722.5000,  3864727.7500,
         3876085.2500,  3878832.5000,  3885635.2500,  3887108.7500,
         3887290.2500,  3893722.0000,  3903814.2500,  3940209.2500,
         3951505.0000,  3964481.7500,  3965860.5000,  3982860.0000,
         3984226

In [ ]:
ipd.Audio(data=wav3.numpy(), autoplay=True, rate=sr)

In [58]:
## Посмотрим на представления wa2vec